In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime 

### 找出要刪除的公司 -> etf, reits 不包含在上市公司之中，刪除金融業即可

In [2]:
fin = pd.read_csv('C:\\Users\\user\\Desktop\\SAS\\financial_list.csv', encoding='unicode-escape',header=None,skiprows=1)
fin = fin[0].str.split(" ",expand=True)[0]
fin = fin.dropna()

In [4]:
[str(i) for i in fin.unique()]

['2801',
 '2809',
 '2812',
 '2816',
 '2820',
 '2823',
 '2832',
 '2834',
 '2836',
 '2838',
 '2845',
 '2849',
 '2850',
 '2851',
 '2852',
 '2867',
 '2880',
 '2881',
 '2882',
 '2883',
 '2884',
 '2885',
 '2886',
 '2887',
 '2888',
 '2889',
 '2890',
 '2891',
 '2892',
 '2897',
 '5876',
 '5880']

### 刪除 dividend 資料裡的 financial id 

In [5]:
def convert(x):
    try:
        x = datetime.strptime(x, '%Y%m%d')
        return x
    except:
        return 0

In [22]:
div = pd.read_csv('C:\\Users\\user\\Desktop\\SAS\\dividend.txt', encoding='unicode-escape', skiprows = 1)
div = div.iloc[:,[0,2,5,6]]
div.columns = ['id','ind','div','ann_date']
print('原始資料大小', div.shape)
print('--------------------------')
print('公司家數', len(div.id.unique()))
print('--------------------------')
print(div.dtypes)
print('==========================')

div.ind = div.ind.astype('category')
div.id = div.id.astype('str')
div = div.set_index('id')
div = div.drop([str(i) for i in fin.unique()], errors = 'ignore')
div = div.reset_index()

print('刪除後資料大小', div.shape)
print('--------------------------')
print('公司家數', len(div.id.unique()))
print('--------------------------')
print(div.dtypes)
print('==========================')

原始資料大小 (6302, 4)
--------------------------
公司家數 763
--------------------------
id            int64
ind           int64
div         float64
ann_date      int64
dtype: object
刪除後資料大小 (6075, 4)
--------------------------
公司家數 732
--------------------------
id            object
ind         category
div          float64
ann_date       int64
dtype: object


In [31]:
div

,id,ind,div,ann_date
0,2201,12,0.505500,20010103
1,1227,2,0.600000,20010112
2,2520,14,2.000000,20010331
3,2546,14,0.600000,20010331
4,2324,25,0.500000,20010419
...,...,...,...,...
6070,9944,20,0.200000,20130813
6071,9927,20,2.200000,20130808
6072,9926,23,1.200000,20130808
6073,9958,10,0.248595,20130913


In [32]:
list = []
for i in div.id.unique():
    tem = div[div['id']==i]
    tem['diff'] = tem['div'] - tem['div'].shift(1)
    list.append(tem)

div = pd.concat(list, ignore_index=True)

# 過濾在2002年之前的宣告日
div = div[div['ann_date']>20020000] 

# 刪除 NAN
div = div.dropna()

# 刪除股利沒增減的資料
div = div[div['diff']!=0]

print('資料大小', div.shape)
print('--------------------------')
print('公司家數', len(div.id.unique()))
print('--------------------------')
print(div.dtypes)

<ipython-input-32-b611133b2127>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tem['diff'] = tem['div'] - tem['div'].shift(1)


資料大小 (4799, 5)
--------------------------
公司家數 682
--------------------------
id            object
ind         category
div          float64
ann_date       int64
diff         float64
dtype: object


In [36]:
div['inc_dec'] = div['diff']>0
div = pd.pivot_table(div,index=['id', 'ann_date'],values=['div','diff','inc_dec'])
print('2002 ~ 2013 股利增加之公司數量', div['inc_dec'].sum())
div

2002 - 2013 股利增加之公司數量 2765.0


diff       div  inc_dec
id   ann_date                             
1101 20040722  0.250000  0.350000      1.0
     20050819  0.350000  0.700000      1.0
     20060727  0.705943  1.405943      1.0
     20070712  0.294057  1.700000      1.0
     20080709  0.200000  1.900000      1.0
...                 ...       ...      ...
9958 20070820 -0.250000  0.500000      0.0
     20080821 -0.300000  0.200000      0.0
     20100825 -0.100000  0.100000      0.0
     20110826  0.100000  0.200000      1.0
     20130913  0.048595  0.248595      1.0

[4797 rows x 3 columns]

In [39]:
print("total dis", div['inc_dec'].sum())
print("total dds", div['inc_dec'].count() - div['inc_dec'].sum())

total dis 2765.0
total dds 2032.0


In [42]:
inc_all =  div['inc_dec'].sum()
dec_all =  div['inc_dec'].count() - inc_all
data1 = []
data2 = []
data3 = []
data4 = []



for i in range(2002,2014):
    inc = (div[(div.index.get_level_values('ann_date')> int(f'{i}0000')) & (div.index.get_level_values('ann_date') < int(f'{i}0000')+10000)]['inc_dec']).sum()
    total =  (div[(div.index.get_level_values('ann_date')> int(f'{i}0000')) & (div.index.get_level_values('ann_date') < int(f'{i}0000')+10000)]['inc_dec']).count()
    
    # 存成df
    data1.append(round(inc))
    data2.append(round(inc / inc_all,4))
    data3.append(round(total - inc))
    data4.append(round( (total - inc) / dec_all ,4))

    # 輸出
    #print(f'{i} 股利增加公司數目 : ', round(inc), 
    #      f', {i} 股利增加公司比率 : ', round(inc / inc_all,4),
    #      f'|| {i} 股利減少公司數目 : ', round(total - inc) , 
    #      f', {i} 股利減少公司比率 : ', round( (total - inc) / dec_all ,4))

df = pd.DataFrame(data={'DIS (number of companies)':data1,
                        'DIS (ratio)':data2, 
                        'DDS (number of companies)':data3,
                        'DDS (ratio)':data4},index = range(2002,2014))
df

,DIS (number of companies),DIS (ratio),DDS (number of companies),DDS (ratio)
2002,95.0,0.0344,91.0,0.0448
2003,192.0,0.0694,71.0,0.0349
2004,225.0,0.0814,110.0,0.0541
2005,248.0,0.0897,141.0,0.0694
2006,246.0,0.0888,150.0,0.0741
2007,284.0,0.1029,148.0,0.0731
2008,288.0,0.1042,182.0,0.0896
2009,79.0,0.0286,330.0,0.1624
2010,323.0,0.1168,125.0,0.0615
2011,351.0,0.1269,158.0,0.0778


In [43]:
df.loc['Total',:] = df.loc['Total',:] = [round(a, 2) for a in df.sum(axis=0).values]
df

,DIS (number of companies),DIS (ratio),DDS (number of companies),DDS (ratio)
2002,95.0,0.0344,91.0,0.0448
2003,192.0,0.0694,71.0,0.0349
2004,225.0,0.0814,110.0,0.0541
2005,248.0,0.0897,141.0,0.0694
2006,246.0,0.0888,150.0,0.0741
2007,284.0,0.1029,148.0,0.0731
2008,288.0,0.1042,182.0,0.0896
2009,79.0,0.0286,330.0,0.1624
2010,323.0,0.1168,125.0,0.0615
2011,351.0,0.1269,158.0,0.0778


In [ ]:
'''

def match(x, y):
    '''
    x 要配對的年度
    y 日期index
    '''
    list = []
    for i in y:
        target = re.compile(x+'-.{2}-.{2}')
        if re.match(target, i):
            list.append(1)
        else:
            list.append(0)
    return list
match('2002',['2002-01-01','2000-01-03'])

'''